# Transactions with [`psycopg2`](http://initd.org/psycopg/docs/)

In this notebook, we showcase how to connect to a [PostgreSQL](https://www.postgresql.org/) database, execute queries, and run transactions with different isolation levels using the [`psycopg2`](http://initd.org/psycopg/docs/) package for python. The notebook is inspired by [this](https://pynative.com/python-postgresql-transaction-management-using-commit-and-rollback/) website.

Copyright Marcel Maltry & Jens Dittrich, [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

# Setup

The following cell serves as setup. We will explain the syntax in more detail below. Here, we simply connect to the database, create a new table `accounts` with attributes `id` and `balance`, and add some toy data.

In [1]:
import psycopg2

# make sure to add your password:
dsn = 'dbname=postgres user=postgres host=/var/run/postgresql/'

def reset_db(dsn):
    # Connect to postgres user's default database
    with psycopg2.connect(dsn) as conn:

        # Open a cursor to perform database operations
        with conn.cursor() as cur:

            # Drop table if existing
            cur.execute("""DROP TABLE IF EXISTS accounts;""")

            # Create accounts table
            cur.execute("""CREATE TABLE accounts
                          (id int PRIMARY KEY, balance float(2));""")

            # Insert sample data into accounts table
            cur.execute("""INSERT INTO accounts
                           VALUES
                               (1, 2000.0),
                               (2, 520.0),
                               (3, 470.0),
                               (4, 1700.0),
                               (5, 2400.0);""")

            # Note: if no exception has been raised by the block, the transaction is commited implicitly.
            
reset_db(dsn)

# Basics

In order to send queries to the database, we first need to establish a `connection`. We call the `connect()` method and provide some basic connection parameters such as the database name, the user, the password, and the host, if the database we try to connect to is running on a remote.

We can send queries to the database with a cursor that has to be opened from an established connection via the `cursor()` method. The cursor allows us both to send queries (`execute()`) and retrieve results (`fetchone()`, `fetchall()`). Results are always tuples, even if they consist of a single integer. We have to consider this when parsing the results. When we are done, we close the cursor (and the connection).

The following example shows how to query the database for an entire table. We also use the cursor to obtain some additional information on the results.

In [2]:
# Connect to database as specified in dsn
conn1 = psycopg2.connect(dsn)

# Set connection to auto-commit mode for now
conn1.autocommit = True

# Open a cursor to perform database operations
cur = conn1.cursor()

# Define a SQL query
q_accounts = """SELECT * FROM accounts;"""

# Execute the query using the cursor
cur.execute(q_accounts)

# Print information on the query and its result
print(f"The last query executed was \"{cur.query}\".")
print(f"The query was executed with status message \"{cur.statusmessage}\".")
print(f"The query returned {cur.rowcount} rows that can be fetched.")
print(f"The cursor currently points to row {cur.rownumber}.")
print(f"The description of the query result is {cur.description}.")

The last query executed was "b'SELECT * FROM accounts;'".
The query was executed with status message "SELECT 5".
The query returned 5 rows that can be fetched.
The cursor currently points to row 0.
The description of the query result is (Column(name='id', type_code=23), Column(name='balance', type_code=700)).


In [3]:
# Fetch results from cursor
accounts = cur.fetchall()

# Print sorted results
print(f"The query returned the following tuples:\n{sorted(accounts)}")

# Close the cursor, connection remains open
cur.close()

The query returned the following tuples:
[(1, 2000.0), (2, 520.0), (3, 470.0), (4, 1700.0), (5, 2400.0)]


# Session Parameters

`psycopg2` allows us to set certain session parameters that handle, how the next transaction of a connection is executed. In the following, we will take a closer look at:

* `readonly`: The session is set to read-only and, thus, write operations will fail with an exception.
* `autocommit`: Every statement sent to the database has an immediate effect, i.e. each statement is an individual transaction that is implicitly commited upon successful completion.
* `isolation_level`: This allows us to choose from multiple isolation levels.

Session parameters can either be set explicitly, e.g. `conn.readonly = True` or using the `set_session()` method provided by connection. Note that session parameters of a connection can only be set if no transaction is currently being performed on the connection.


## Read-Only

If we set a connection to read-only mode, write operations will not be executed and instead will raise an `ReadOnlySqlTransaction` exception. The following example demonstrates this behavior.

In [4]:
# Set connection to read-only mode
conn1.readonly = True

# Open a cursor to perform database operations
cur = conn1.cursor()

try:
    # Try to insert a new tuple into the table
    cur.execute("INSERT INTO accounts VALUES (5, 100000.0);")
    
    # If successful, print newly added tuple
    cur.execute("SELECT * FROM accounts WHERE id=5;")
    print(cur.fetchone())

except psycopg2.errors.ReadOnlySqlTransaction:
    print(f"ERROR: The query failed due to the connection being read-only.")

finally:
    # Close the cursor, connection remains open
    cur.close()
    
    # Set connection to read-write mode
    conn1.readonly = False

ERROR: The query failed due to the connection being read-only.


## Auto-Commit

If `autocommit` is set to `True`, each call of `cur.execute()` is handled as an individual transactions and will either have an immediate effect or fail. The following example shows that each modification is immediately visible to other connections to the database.

In [5]:
# Set connection to auto-commit mode
conn1.autocommit = True

# Open a second connection to the database in read-only and auto-commit mode
conn2 = psycopg2.connect(dsn)
conn2.set_session(readonly=True, autocommit=True)

# Open a cursor on second connection to perform database operations
cur2 = conn2.cursor()

# Open a cursor on the first (auto-commit) connection and insert a new tuple
cur1 = conn1.cursor()
cur1.execute("""INSERT INTO accounts VALUES (6, 237.0);""")

# Use cursor from second connection to see immediate effect
cur2.execute("""SELECT * FROM accounts WHERE id=6;""")
print(f"Due to auto-commit, the tuple is already visible to the other connection: {cur2.fetchall()}")

# Close both cursors, connections remain open
cur1.close()
cur2.close()

Due to auto-commit, the tuple is already visible to the other connection: [(6, 237.0)]


### Commit Transaction

If we set `autocommit` to `False` (this is the default setting), the first call of execute on a cursor begins a new transaction ([`BEGIN`](https://www.postgresql.org/docs/current/sql-begin.html)) and `commit()` has to be called explicitly to make the transaction persistent ([`COMMIT`](https://www.postgresql.org/docs/current/sql-commit.html)). The example below transfers money from one account to another. It is equivalent to running the following transaction directly from the database shell:
```SQL
BEGIN;
UPDATE accounts SET balance = balance - 100 WHERE id=3;
UPDATE accounts SET balance = balance + 100 WHERE id=1;
COMMIT;
```
We also show that as long as the transaction is not commited, changes are not visible to other connections. Note that this also depends on the isolation level (we will get back to this).

### Show visibility of local changes:

In [6]:
# Set connection to transaction mode
conn1.autocommit = False

# Open a cursor to perform database operations
cur1 = conn1.cursor()

# Open a cursor on second connection to perform database operations
cur2 = conn2.cursor()

# Update balance of account 3, implicitly begins a transaction
cur1.execute("""UPDATE accounts SET balance = balance - 100 WHERE id=3;""")

# Update balance of account 1, implicitly begins a transaction
cur1.execute("""UPDATE accounts SET balance = balance + 100 WHERE id=1;""")

# Compare states visible to both transactions
q_acc = """SELECT * FROM accounts WHERE id=1 OR id=3;"""
cur1.execute(q_acc)
cur2.execute(q_acc)
print(f"Account balances observed by each connection before COMMIT:\n"\
      f"Transaction 1: {cur1.fetchall()}\n"\
      f"Transaction 2: {cur2.fetchall()}\n"\
      f"Changes not yet visible to connection 2."\
     )

Account balances observed by each connection before COMMIT:
Transaction 1: [(1, 2100.0), (3, 370.0)]
Transaction 2: [(1, 2000.0), (3, 470.0)]
Changes not yet visible to connection 2.


In [7]:
# Commit transaction
conn1.commit()
print("--Transaction commited--")

# Compare states visible to both transactions
cur1.execute(q_acc)
cur2.execute(q_acc)
print(f"Account balances observed by each connection after COMMIT:\n"\
      f"Transaction 1: {cur1.fetchall()}\n"\
      f"Transaction 2: {cur2.fetchall()}\n"\
      f"Changes visible to transaction 2."\
     )

# close both cursors and commit running transaction of connection 1
cur1.close()
conn1.commit()
cur2.close()

--Transaction commited--
Account balances observed by each connection after COMMIT:
Transaction 1: [(1, 2100.0), (3, 370.0)]
Transaction 2: [(1, 2100.0), (3, 370.0)]
Changes visible to transaction 2.


### Rollback Transactions

The next example shows a similar transaction as above. The only difference is that instead of making the changes persistent, we decide to [`ABORT`](https://www.postgresql.org/docs/current/sql-abort.html) the transaction by calling `rollback()` on the connection. It is equivalent to running the following transaction directly from the database shell:
```SQL
BEGIN;
UPDATE accounts SET balance = balance - 100 WHERE id=3;
UPDATE accounts SET balance = balance + 100 WHERE id=1;
ABORT;
```
All changes performed by the aborted must not become durable in the database. Note that if we `close()` an open connection, `rollback()` will be performed implicitly. In other words: if `autocommit` is set to `False`, calling `close()` is equivalent to aborting both the transaction and closing the connection.

In [8]:
# Open a cursor to perform database operations
cur1 = conn1.cursor()

# Open a cursor on second connection to perform database operations
cur2 = conn2.cursor()

# Update balance of account 3, implicitly begins a transaction
cur1.execute("""UPDATE accounts SET balance = balance - 100 WHERE id=3;""")

# Update balance of account 1, implicitly begins a transaction
cur1.execute("""UPDATE accounts SET balance = balance + 100 WHERE id=1;""")

# Compare states visible to both transactions
q_acc = """SELECT * FROM accounts WHERE id=1 OR id=3;"""
cur1.execute(q_acc)
cur2.execute(q_acc)
print(f"Account balances observed by each transaction before COMMIT:\n"\
      f"Transaction 1: {cur1.fetchall()}\n"\
      f"Transaction 2: {cur2.fetchall()}\n"\
      f"Changes not visible to transaction 2."\
     )

Account balances observed by each transaction before COMMIT:
Transaction 1: [(1, 2200.0), (3, 270.0)]
Transaction 2: [(1, 2100.0), (3, 370.0)]
Changes not visible to transaction 2.


In [9]:
# Abort transaction
conn1.rollback()
print("--Transaction aborted--")

# Compare states visible to both transactions
cur1.execute(q_acc)
cur2.execute(q_acc)
print(f"Account balances observed by each connection after COMMIT:\n"\
      f"Transaction 1: {cur1.fetchall()}\n"\
      f"Transaction 2: {cur2.fetchall()}\n"\
      f"Changes of transaction undone."\
     )

# close both cursors and commit running transaction of connection 1
cur1.close()
conn1.commit()
cur2.close()

--Transaction aborted--
Account balances observed by each connection after COMMIT:
Transaction 1: [(1, 2100.0), (3, 370.0)]
Transaction 2: [(1, 2100.0), (3, 370.0)]
Changes of transaction undone.


## Isolation Levels

Furthermore, we can set the isolation level ([`SET TRANSACTION`](https://www.postgresql.org/docs/current/sql-set-transaction.html)) per session either by calling `set_isolation_level()` or `set_session()` with the corresponding parameters.

The following example showcases the impact of isolation levels. While transaction 1 withdraws money from a bank account, transaction 2 sets its isolation level to `REPEATABLE READ` and reads the entry for this bank account. After transaction 1 commited, transaction 2 again accesses the same bank account. However, since its isolation level is set to `REPEATABLE READ` it still sees the unchanged data.

The scenario is equivalent to running the following transactions in parallel from two database shells.

Transaction 1:
```SQL
BEGIN;
SELECT * FROM accounts WHERE id=2;
UPDATE accounts SET balance = balance - 50 WHERE id=2;
COMMIT;
```

Transaction 2:
```SQL
BEGIN;
SET TRANSACTION ISOLATION LEVEL REPEATABLE READ;
SELECT * FROM accounts WHERE id=2;
-- in the meantime Transaction 1 updates account 2 and commits.
SELECT * FROM accounts WHERE id=2;
COMMIT;
```

Note that in SQL, isolation levels are set *within* a transaction block while `psycopg2` requires us to set the isolation level *before* we start a new transaction. The isolation level supported by PostgreSQL and `psycopg2` can be found [here](http://initd.org/psycopg/docs/extensions.html#isolation-level-constants).

In [10]:
# Set isolation levels
# recall that the default isolation level in PostgreSQL is READ COMMITTED
conn1.set_session(isolation_level=psycopg2.extensions.ISOLATION_LEVEL_DEFAULT,
                  readonly=False, autocommit=False)
conn2.set_session(isolation_level=psycopg2.extensions.ISOLATION_LEVEL_REPEATABLE_READ,
                  readonly=False, autocommit=False)
#conn2.set_session(isolation_level=psycopg2.extensions.ISOLATION_LEVEL_READ_COMMITTED,
#                  readonly=False, autocommit=False)


# Open cursors for both connections to perform database operations
cur1 = conn1.cursor()
cur2 = conn2.cursor()

# Compare states visible to both transactions
q_acc = """SELECT * FROM accounts WHERE id=2;"""
cur1.execute(q_acc)
cur2.execute(q_acc)
print(f"Account balance observed by each transaction:\n"\
      f"Transaction 1: {cur1.fetchall()}\n"\
      f"Transaction 2: {cur2.fetchall()}\n"\
      f"Both transactions see the same balance.")

Account balance observed by each transaction:
Transaction 1: [(2, 520.0)]
Transaction 2: [(2, 520.0)]
Both transactions see the same balance.


In [11]:
# Withdraw money from account 2 and commit
cur1.execute("""UPDATE accounts SET balance = balance - 50 WHERE id=2;""")
conn1.commit()
print("--update perfomed, transaction 1 commited--")

# Compare states visible to both transactions
q_acc = """SELECT * FROM accounts WHERE id=2;"""
cur1.execute(q_acc)
cur2.execute(q_acc)
print(f"Account balance observed by each transaction:\n"\
      f"Transaction 1: {cur1.fetchall()}\n"\
      f"Transaction 2: {cur2.fetchall()}\n"\
      f"Transaction 2 still sees the state from the beginning of the transaction.")

# Close both cursors and connections
cur1.close()
cur2.close()
conn1.close()
conn2.close()  # performs implicit rollback

--update perfomed, transaction 1 commited--
Account balance observed by each transaction:
Transaction 1: [(2, 470.0)]
Transaction 2: [(2, 520.0)]
Transaction 2 still sees the state from the beginning of the transaction.


# Exercises

In the exercises, we will implement an execution plan with three simultaneous transactions as well as a function for transferring money safely.

## Exercise 1 (Execution Plan)

Implement the execution plan from the assignment sheet. Whenever a transaction reads all tuples, also print the tuples in a sorted manner and label them with the transaction, e.g. `print(f"T1: {sorted(cur1.fetchall())}")`.

Note that in order for the execution plan to run without any problems, `autocommit=True` is needed as in all other cases by simply executing the plan, locks cannot be acquired and the cell will not terminate. You should investigate the impact of the different isolation levels in the second part of the exercise.

In [12]:
# Reset db
# This ensures that all changes before do not influence your results.
dsn = 'dbname=postgres user=postgres host=/var/run/postgresql/'
reset_db(dsn)

# Insert your implementation of the execution plan here

## Exercise 2 (Money Transfer)

Implement the function `transfer_money()` that transfers money from one account to another. For each transfer there is a fee that should also be transferred from the source account to the bank's account (id 0). The transaction should only be executed, if all involved accounts exist and the source account has sufficient money. Otherwise, the transaction should be aborted and an exception should be raised. All checks and the transfer should be executed in a single transaction. Choose a suitable isolation level for this application. Use the provided unit test to debug your implementation.

In [13]:
def transfer_money(conn, src, dst, amount, fee):
    """
    Transfers amount of money from src account to dst account via connection conn.
    The src account must also pay a transaction fee to the bank's account.
    The transaction is only executed if all accounts exist and the src account
    has enough money, otherwise it is rolled back.
    
    Params:
        conn: pycopg2 connection object the transaction should be performed on
        src: id of the source account
        dst: id of the destination account
        amount: amount of money that should be transferred
        fee: percentage of transfer fee that should be transferred to bank's account (id=0)
    """
    # Insert your implementation here
    pass

In [14]:
import unittest


class TestMoneyTransfer(unittest.TestCase):
    
    def setUp(self):
        dsn = 'dbname=postgres user=postgres host=/var/run/postgresql/'
        
        # Connect to database
        self.conn = psycopg2.connect(dsn)
        self.conn.set_session(autocommit=True)
        
        # Obtain cursor
        cur = self.conn.cursor()
        
        # Drop table if existing
        cur.execute("""DROP TABLE IF EXISTS accounts;""")
        
        # Create new table
        cur.execute("""CREATE TABLE accounts (id int PRIMARY KEY, balance float(2));""")
        
        # Insert sample data into accounts table
        cur.execute("""INSERT INTO accounts VALUES
                    (0, 0.0),
                    (1, 2000.0),
                    (2, 520.0),
                    (3, 470.0),
                    (4, 1700.0),
                    (5, 2400.0);""")       
    
    def tearDown(self):
        # Obtain cursor
        cur = self.conn.cursor()
        
        # Drop table
        cur.execute("""DROP TABLE IF EXISTS accounts;""")
        self.conn.commit()
        
        # Close connection and cursor
        cur.close()
        self.conn.close()
    
    def test_sessionparamas(self):
        transfer_money(self.conn, 1, 2, 100, 0.01)
        
        # Check autocommit
        self.assertFalse(self.conn.autocommit,
                         msg="Autocommit should be set to false.")
        # Check readonly
        self.assertFalse(self.conn.readonly,
                         msg="Readonly should be set to false.")
        # Check isolation level
        self.assertEqual(self.conn.isolation_level,
                         psycopg2.extensions.ISOLATION_LEVEL_SERIALIZABLE,
                         msg="Isolation level should be SERIALIZABLE.")
        
    def test_accountexist(self):
        with self.assertRaises(Exception,
                               msg="Source account does not exist, exception should be raised"):
            transfer_money(self.conn, 6, 2, 100, 0.01)
            
        with self.assertRaises(Exception,
                               msg="Destination account does not exist, exception should be raised"):
            transfer_money(self.conn, 1, 7, 100, 0.01)
            
        with self.assertRaises(Exception,
                               msg="Bank's account does not exist, exception should be raised"):
            # Delete bank's account
            cur = self.conn.cursor()
            cur.execute("""DELETE FROM accounts WHERE id=0;""")
            cur.close()
            conn.commit()
            
            transfer_money(self.conn, 1, 3, 100, 0.01)
            
    def test_sufficientmoney(self):
        # Check amount too high
        with self.assertRaises(Exception,
                               msg="Source account has insufficient money, exception should be raised"):
            transfer_money(self.conn, 5, 2, 2401, 0.01)
        # Check amount+fee too high
        with self.assertRaises(Exception,
                               msg="Source account has insufficient money, exception should be raised"):
            transfer_money(self.conn, 5, 2, 2353, 0.02)     
        
    def test_correcttransfer(self):
        transfer_money(self.conn, 3, 4, 234, 0.05)
        cur = self.conn.cursor()
        q_bal = """SELECT balance FROM accounts WHERE id=%s;"""
        
        cur.execute(q_bal, (3,))
        acc3 = cur.fetchone()[0]
        cur.execute(q_bal, (4,))
        acc4 = cur.fetchone()[0]
        cur.execute(q_bal, (0,))
        bank = cur.fetchone()[0]
        
        self.assertEqual(acc3, round(470-234-234*0.05, 2),
                         msg="Source account balance incorrect.")
        self.assertEqual(acc4, 1700+234,
                         msg="Destination account balance incorrect.")
        self.assertEqual(bank, 0+round(234*0.05, 2),
                         msg="Bank's account balance incorrect.")
    

In [15]:
# Run the unit test without shutting down the jupyter kernel
unittest.main(argv=['ignored', '-v'], verbosity=2, exit=False)

test_accountexist (__main__.TestMoneyTransfer) ... ERROR
test_correcttransfer (__main__.TestMoneyTransfer) ... ERROR
test_sessionparamas (__main__.TestMoneyTransfer) ... ERROR
test_sufficientmoney (__main__.TestMoneyTransfer) ... ERROR

ERROR: test_accountexist (__main__.TestMoneyTransfer)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-ac2bcd6d030f>", line 10, in setUp
    self.conn = psycopg2.connect(dsn)
  File "/home/vagrant/.conda/envs/bde/lib/python3.7/site-packages/psycopg2/__init__.py", line 127, in connect
    conn = _connect(dsn, connection_factory=connection_factory, **kwasync)
psycopg2.OperationalError: could not connect to server: No such file or directory
	Is the server running locally and accepting
	connections on Unix domain socket "/tmp/.s.PGSQL.5432"?


ERROR: test_correcttransfer (__main__.TestMoneyTransfer)
----------------------------------------------------------------------
Traceb